In [2]:
#create a Spark Session
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('Missing').getOrCreate()

In [4]:
# Read the dataset
training=spark.read.csv('test1.csv', header='True', inferSchema='True')

In [5]:
training.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [6]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [7]:
training.columns

['Name', 'age', 'Experience', 'Salary']

In [26]:
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=["age", "Experience"], outputCol="independent feature")

In [27]:
output=featureassembler.transform(training)

In [28]:
output.show()

+---------+---+----------+------+-------------------+
|     Name|age|Experience|Salary|independent feature|
+---------+---+----------+------+-------------------+
|    Krish| 31|        10| 30000|        [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|         [30.0,8.0]|
|    Sunny| 29|         4| 20000|         [29.0,4.0]|
|     Paul| 24|         3| 20000|         [24.0,3.0]|
|   Harsha| 21|         1| 15000|         [21.0,1.0]|
|  Shubham| 23|         2| 18000|         [23.0,2.0]|
+---------+---+----------+------+-------------------+



In [30]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'independent feature']

In [39]:
finalized_data=output.select('independent feature','Salary')

In [40]:
finalized_data.show()

+-------------------+------+
|independent feature|Salary|
+-------------------+------+
|        [31.0,10.0]| 30000|
|         [30.0,8.0]| 25000|
|         [29.0,4.0]| 20000|
|         [24.0,3.0]| 20000|
|         [21.0,1.0]| 15000|
|         [23.0,2.0]| 18000|
+-------------------+------+



In [36]:
x=output.select('age', 'Experience')

In [37]:
x.show()

+---+----------+
|age|Experience|
+---+----------+
| 31|        10|
| 30|         8|
| 29|         4|
| 24|         3|
| 21|         1|
| 23|         2|
+---+----------+



In [41]:
finalized_data.show()

+-------------------+------+
|independent feature|Salary|
+-------------------+------+
|        [31.0,10.0]| 30000|
|         [30.0,8.0]| 25000|
|         [29.0,4.0]| 20000|
|         [24.0,3.0]| 20000|
|         [21.0,1.0]| 15000|
|         [23.0,2.0]| 18000|
+-------------------+------+



In [56]:
#Applying Liner Regression
from pyspark.ml.regression import LinearRegression
train_data, test_data=finalized_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol='independent feature', labelCol='Salary')
regressor=regressor.fit(train_data)

In [57]:
### Coefficients
regressor.coefficients

DenseVector([-383.9733, 1711.1853])

In [58]:

### Intercepts
regressor.intercept

23998.330550919385

In [60]:
### Prediction
predict=regressor.evaluate(test_data)

In [61]:
predict.predictions.show()

+-------------------+------+------------------+
|independent feature|Salary|        prediction|
+-------------------+------+------------------+
|         [21.0,1.0]| 15000| 17646.07679465793|
|         [23.0,2.0]| 18000|18589.315525876566|
+-------------------+------+------------------+



In [64]:
predict.meanAbsoluteError, predict.meanSquaredError, predict.degreesOfFreedom

(1617.6961602672473, 3674507.5961331767, -1)